IMAGE CLASSIFICATION TASK ON KAGGLE:

DATA PREPROCESSING:

In [ ]:
# data preprocessing (includes image localization):
from PIL import Image
import os
classes=[]
for i in os.listdir('train'):
    classes.append(i)
    annotation=open("train/"+i+"/"+i+"_boxes.txt")
    for line in annotation:
        spec=line.split()
        if int(spec[1])==int(spec[3]) or int(spec[2])==int(spec[4]):
            pass
        else:
            image = Image.open('train/'+i+'/images/'+spec[0])
            cropped = image.crop((int(spec[1]), int(spec[2]), int(spec[3]), int(spec[4])))
            cropped.save('train/'+i+'/images/'+spec[0]) #to avoid localizing use image.save instead of cropped.save
    annotation.close()
for i in os.listdir('train'):
    os.remove("train/"+i+"/"+i+"_boxes.txt")
import shutil
for i in os.listdir('train'):
    source = 'train/'+i+'/images/'
    dest = 'train/'+i

    files = os.listdir(source)

    for f in files:
        shutil.move(source+f, dest)
for i in os.listdir('train'):
    os.rmdir('train/'+i+'/images')

In [ ]:
import os
from PIL import Image
val_annotations=open('val/val_annotations.txt')
for lines in val_annotations:
    val_list=lines.split()
    isdir = os.path.exists('val/'+val_list[1])
    if not(isdir):
        os.mkdir('val/'+val_list[1])
    image = Image.open('val/images/'+val_list[0])
    if int(val_list[2])==int(val_list[4]) or int(val_list[3])==int(val_list[5]):
        image.save('val/'+val_list[1]+'/'+val_list[0])
    else:
        cropped = image.crop((int(val_list[2]), int(val_list[3]), int(val_list[4]), int(val_list[5])))
        cropped.save('val/'+val_list[1]+'/'+val_list[0]) #to avoid localizing use image.save instead of cropped.save
val_annotation.close()


In [ ]:
#Data preprocessing(without image localization):
import numpy as np
from PIL import Image
import os
train=[]
y_train=[]
po=0
for i in os.listdir('train'):
    for j in range(450):
        image = Image.open('train/'+i+'/images/'+i+'_'+str(j)+'.jpeg')
        img_arr = np.array(image)
        if img_arr.shape==(64, 64, 3):
            train.append(img_arr)
            a=np.zeros(200)
            a[po]=1
            y_train.append(a)
        elif img_arr.shape==(3, 64, 64):
            img_arr=np.transpose(img_arr,(1,2,0))
            train.append(img_arr)
            a=np.zeros(200)
            a[po]=1
            y_train.append(a)
        elif img_arr.shape==(64, 64):
            jk=np.array([img_arr,img_arr,img_arr]) #adding 2 more channels to make gray scale images similar to RGB images
            img_arr=np.transpose(jk,(1,2,0))
            train.append(img_arr)
            a=np.zeros(200)
            a[po]=1
            y_train.append(a)
        else:
            break
    po+=1
t=np.array(train)
yt=np.array(y_train)
np.save('xtrain.npy',t)
np.save('ytrain.npy',yt)

In [ ]:
from PIL import Image
import numpy as np
import os
val=[]
y_val=[]
ro=0
for i in os.listdir('val'):
    for j in os.listdir('val/'+i):
        image = Image.open('val/'+i+'/'+j)
        img_arr = np.array(image)
        if img_arr.shape==(64, 64, 3):
            val.append(img_arr)
            a=np.zeros(200)
            a[ro]=1
            y_val.append(a)
        elif img_arr.shape==(3, 64, 64):
            img_arr=np.transpose(img_arr,(1,2,0))
            val.append(img_arr)
            a=np.zeros(200)
            a[ro]=1
            y_val.append(a)
        elif img_arr.shape==(64, 64):
            jk=np.array([img_arr,img_arr,img_arr])#adding 2 more channels to make gray scale images similar to RGB images
            img_arr=np.transpose(jk,(1,2,0))
            val.append(img_arr)
            a=np.zeros(200)
            a[ro]=1
            y_val.append(a)
        else:
            break
    ro+=1
v=np.array(val)
yv=np.array(y_val)
np.save('xval.npy',v)
np.save('yval.npy',yv)

In [ ]:
from PIL import Image
import numpy as np
import os
val=[]
# y_val=[]
# ro=0
for j in kl:
    print(j)
    j='test_'+str(j)+'.JPEG'
    image = Image.open('test/images/'+j)
    t=j.split('_')
    print(t[1])
    img_arr = np.array(image)
    if img_arr.shape==(64, 64, 3):
#         print(int(t[1].split('.')[0]))
        val.append(img_arr)
#         pass
#         print(img_arr.shape)
    elif img_arr.shape==(3, 64, 64):
        img_arr=np.transpose(img_arr,(1,2,0))
        print(int(t[1].split('.')[0]))
        val.append(img_arr)
#         image.show()
    elif img_arr.shape==(64, 64):
        jk=np.array([img_arr,img_arr,img_arr])#adding 2 more channels to make gray scale images similar to RGB images
        img_arr=np.transpose(jk,(1,2,0))
#         print(t.shape)
#         img = Image.fromarray(t, 'RGB')
#         img.show()
        print(int(t[1].split('.')[0]))
        val.append(img_arr)
    else:
        break
v=np.array(val)
# print(v.shape)
np.save('xtest.npy',v)

EXPERIMENT 1:

In [ ]:
#required libraries:
import numpy as np
import os
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten, Dropout, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint

#Loading data image data generator:
trdata = ImageDataGenerator()
traindata = trdata.flow_from_directory(directory=r"/kaggle/input/imagedetect/train/",target_size=(224,224))
tsdata = ImageDataGenerator()
testdata = tsdata.flow_from_directory(directory=r"/kaggle/input/imagedetect/val/", target_size=(224,224))
#model implementation:
model = Sequential()
model.add(Conv2D(input_shape=(224,224,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
# model.add(BatchNormalization())
model.add(Flatten())
# model.add(Dropout(.5))
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=4096,activation="relu"))
# model.add(Dropout(.5))
model.add(Dense(units=2, activation="softmax"))

from keras.optimizers import Adam
optimize = Adam(lr=0.001)
model.compile(optimizer=optimize, loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])
callbacks=[ModelCheckpoint(filepath='model_{epoch}',save_freq='epoch')]
model.fit_generator(steps_per_epoch=100,generator=traindata, validation_data= testdata, validation_steps=10,epochs=20,callbacks=callbacks)
model.save("my_model")

EXPERIMENT 2:

In [ ]:
#required libraries:
import tensorflow.keras
from tensorflow.keras.layers import Dense, Conv2D
from tensorflow.keras.layers import BatchNormalization, Activation
from tensorflow.keras.layers import AveragePooling2D, Input, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
# from tensorflow.keras.callbacks import LearningRateScheduler
# from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2
# from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
import numpy as np
import os
#Loading data as npy arrays:
x_train = np.load('../input/npyformat/xtrain.npy')
x_val = np.load('../input/npyformat/xval.npy')
y_train = np.load('../input/npyformat/ytrain.npy')
y_val = np.load('../input/npyformat/yval.npy')
X_test = np.load('../input/testing/xtest.npy')
X_test = X_test.astype('float32')/255.
# Input image dimensions.
input_shape = x_train.shape[1:]    
DEPTH = 3 * 9 + 2
# Normalize data.
x_train = x_train.astype('float32') / 255.
x_val = x_val.astype('float32') / 255.

#Pixel Normalization:     
x_train_mean = np.mean(x_train, axis=0)
x_train -= x_train_mean
x_val -= x_train_mean
#model implementation:
model = resnet_model(input_shape=input_shape, depth=DEPTH)
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.001),
              metrics=['accuracy'])
def res_layers(inputs,
                 num_filters=16,
                 kernel_size=3,
                 strides=1,
                 activation='relu',
                 batch_normalization=True,
                 conv_first=True):
    conv = Conv2D(num_filters,
                  kernel_size=kernel_size,
                  strides=strides,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4))
    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
    else:
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
        x = conv(x)
    return x

def resnet_model(input_shape, depth, num_classes=200):
    if (depth - 2) % 9 != 0:
        raise ValueError('depth should be 9n+2 (eg 56 or 110 in [b])')
    num_filters_in = 16
    num_res_blocks = int((depth - 2) / 9)
    inputs = Input(shape=input_shape)
    x = res_layers(inputs=inputs,
                     num_filters=num_filters_in,
                     conv_first=True)
    for stage in range(3):
        for res_block in range(num_res_blocks):
            activation = 'relu'
            batch_normalization = True
            strides = 1
            if stage == 0:
                num_filters_out = num_filters_in * 4
                if res_block == 0:  
                    activation = None
                    batch_normalization = False
            else:
                num_filters_out = num_filters_in * 2
                if res_block == 0:  
                    strides = 2   
            y = res_layers(inputs=x,
                             num_filters=num_filters_in,
                             kernel_size=1,
                             strides=strides,
                             activation=activation,
                             batch_normalization=batch_normalization,
                             conv_first=False)
            y = res_layers(inputs=y,
                             num_filters=num_filters_in,
                             conv_first=False)
            y = res_layers(inputs=y,
                             num_filters=num_filters_out,
                             kernel_size=1,
                             conv_first=False)
            if res_block == 0:
                x = res_layers(inputs=x,
                                 num_filters=num_filters_out,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = tensorflow.keras.layers.add([x, y])

        num_filters_in = num_filters_out
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = AveragePooling2D(pool_size=8)(x)
    y = Flatten()(x)
    outputs = Dense(num_classes,
                    activation='softmax',
                    kernel_initializer='he_normal')(y)
    model = Model(inputs=inputs, outputs=outputs)
    return model

callbacks=[ModelCheckpoint(filepath='model_{epoch}',save_freq='epoch')]
datagen = ImageDataGenerator(
        zca_epsilon=1e-06,
        rotation_range=0,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.,
        zoom_range=0.,
        channel_shift_range=0.,
        fill_mode='nearest',
        cval=0.,
        horizontal_flip=True,
        vertical_flip=False,
        rescale=None,
        preprocessing_function=None,
        data_format=None,
        validation_split=0.0)
datagen.fit(x_train)
model.fit(datagen.flow(x_train, y_train, 
                        batch_size=128),
                        validation_data=(x_val, y_val),
                        epochs=50, verbose=1, workers=1,
                        callbacks=callbacks, 
                        use_multiprocessing=False)
model.save("my_model")

EXPERIMENT 3:

In [ ]:
#required libraries:
import numpy as np
import pandas as pd
# from sklearn.utils.multiclass import unique_labels
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
import itertools
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import confusion_matrix
from keras import Sequential
from tensorflow.keras import models
from keras.applications import VGG19 #For Transfer Learning
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD,Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau
from keras.layers import Flatten,Dense,BatchNormalization,Activation,Dropout
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import LearningRateScheduler
#Loading data as npy arrays:
x_train = np.load('../input/npyformatz/xtrain.npy')
x_val = np.load('../input/npyformats/xval.npy')
y_train = np.load('../input/npyformats/ytrain.npy')
y_val = np.load('../input/npyformats/yval.npy')
x_train = x_train.astype('float32')
x_val = x_val.astype('float32')
#Normalize data:
x_train /= 255.
x_val /= 255.
x_test = np.load('../input/npyformats/xtest.npy')
x_test=x_test.astype('float32')
x_test/=255.
#Pixel Normalization:
x_train_mean = np.mean(x_train, axis=0)
x_train -= x_train_mean
x_val -= x_train_mean
#Image Data Augmentation
train_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True, zoom_range=.1)

val_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True, zoom_range=.1)

test_generator = ImageDataGenerator(rotation_range=2, horizontal_flip= True, zoom_range=.1)

#Fitting the augmentation defined above to the data
train_generator.fit(x_train)
val_generator.fit(x_val)
test_generator.fit(x_test)
#model implementation:
def lr_schedule(epoch):
    """
    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 25:
        lr *= 0.5e-3
    elif epoch > 20:
        lr *= 1e-3
    elif epoch > 15:
        lr *= 1e-12
    elif epoch > 5:
        lr *= 1e-1
    return lr
lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-3)
#Defining the VGG Convolutional Neural Net
base_model = VGG19(include_top = False, weights = 'imagenet', input_shape = (64,64,3), classes = y_train.shape[1])
for layer in base_model.layers:
    layer.trainable=False
base_model.summary()
model= Sequential()
model.add(base_model)
model.add(BatchNormalization())
model.add(Flatten())
model.summary()
model.add(Dropout(.5))
model.add(Dense(1024,activation=('relu'),input_dim=512))
# model.add(Dense(512,activation=('relu'))) 
model.add(Dropout(.5))
# model.add(Dense(256,activation=('relu'))) 
# model.add(Dense(128,activation=('relu')))
model.add(Dense(200,activation=('softmax'))) 
#Initializing the hyperparameters
batch_size= 32
learn_rate=.001
sgd=SGD(lr=learn_rate,momentum=.9,nesterov=False)
adam=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
validation_data = val_generator.flow(x_val, y_val, batch_size = batch_size)
# callbacks=[ModelCheckpoint(
#           filepath='model_{epoch}',
#           save_freq='epoch')]
callbacks=[lr_reducer,lr_scheduler]

model.compile(optimizer=sgd,loss='categorical_crossentropy',metrics=['accuracy'])
model.fit_generator(train_generator.flow(x_train, y_train, batch_size= batch_size),epochs = 14, validation_data = validation_data, callbacks=callbacks, verbose = 1)
for layer in base_model.layers:
    layer.trainable=True
model.summary()
full_model = Sequential(layers=model.layers)
full_model.compile(optimizer=sgd,loss='categorical_crossentropy',metrics=['accuracy'])
full_model.fit_generator(train_generator.flow(x_train, y_train, batch_size= batch_size),epochs = 30, validation_data = validation_data, callbacks=callbacks, verbose = 1)
model.save("my_model")

PREDICTION AND OUTPUT PROCESSING:

In [ ]:
#result prediction:
loaded_model = models.load_model("my_model")
results=model.predict(x_test)
pred=results.argmax(axis=1)
output=open('prediction.txt','w')
for i in pred:
    output.write(i)
    output.write('\n')
output.close()

post=open('prediction.txt')
classess=[]
truth=[]
for i in os.listdir('train'):
    classess.append(i)
for j in post:
    truth.append(classess[int(j)])
# print(truth)
post.close()

linez=[]
j=0
for i in range(10000):
    k='test_'+str(i)+'.JPEG'
    t=truth[j]
    j+=1
    l=[k,t]
    linez.append(','.join(l))
# print(j)
# print(len(linez))

filename="prediction.csv"
headers="file_name,category\n"
f=open(filename,"w",encoding='utf-8')
f.write(headers)
for i in linez:
    f.write(i+'\n')
f.close()